In [109]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation

# Number of bodies
N = 7

# Astronomical units
distance_range_low = 0.4
distance_range_high = 40

# Units of earth's mass
mass_range_low = 0.055
mass_range_high = 20

planet_positions_initial = np.random.uniform(low=distance_range_low, high=distance_range_high, size=(N, 3))
planet_masses = np.random.uniform(low=mass_range_low, high=mass_range_high, size=N)

time_steps_count = 10000  # Reduced for quicker animation generation
time_end = 100
delta_t = time_end / time_steps_count
time_steps = np.linspace(0, time_end, num=time_steps_count)

epsilon = 0.001
G = 6.67430e-11

print_every = round(time_steps_count * 0.05)



In [110]:
positions = np.zeros((time_steps_count, N, 3))
velocities = np.zeros((time_steps_count, N, 3))

positions[0] = planet_positions_initial


for time_step in range(1, len(time_steps)):

    velocities_n = np.zeros((N, 3))
    positions_n = np.zeros((N, 3))
    
    for planet_index in range(0, N):
        mask = np.ones(N)
        mask[planet_index] = 0

        relative_distances = positions[time_step - 1, planet_index] - positions[time_step - 1]
        # Avoid numerical error of division by 0
        relative_distance_sizes = np.linalg.norm(relative_distances, axis=1) + epsilon
        relative_distances_unit = relative_distances / relative_distance_sizes[:, np.newaxis]

        denominator = 1 / (relative_distance_sizes ** 2)
        forces = relative_distances_unit * mask[:, np.newaxis] * G * planet_masses[:, np.newaxis] * denominator[:, np.newaxis]

        velocities_n[planet_index] = forces.sum(axis=0) * delta_t + velocities[time_step - 1, planet_index]
        positions_n[planet_index] = positions[time_step - 1, planet_index] + velocities_n[planet_index] * delta_t

    positions[time_step] = positions_n
    velocities[time_step] = velocities_n

    if time_step % print_every == 0:
        print('Processed', 100 * time_step / time_steps_count, 'Percent')

Processed 5.0 Percent
Processed 10.0 Percent
Processed 15.0 Percent
Processed 20.0 Percent
Processed 25.0 Percent
Processed 30.0 Percent
Processed 35.0 Percent
Processed 40.0 Percent
Processed 45.0 Percent
Processed 50.0 Percent
Processed 55.0 Percent
Processed 60.0 Percent
Processed 65.0 Percent
Processed 70.0 Percent
Processed 75.0 Percent
Processed 80.0 Percent
Processed 85.0 Percent
Processed 90.0 Percent
Processed 95.0 Percent


In [ ]:
# Plotting and Animation
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Create planet points and trails for all planets
points = [ax.plot([], [], [], 'o', label=f'Planet {i+1}')[0] for i in range(N)]
trails = [ax.plot([], [], [], '-', alpha=0.5)[0] for i in range(N)]

def init():
    for point, trail in zip(points, trails):
        point.set_data([], [])
        point.set_3d_properties([])
        trail.set_data([], [])
        trail.set_3d_properties([])
    return points + trails

def update(frame):
    for i in range(N):
        # Update planet position (as arrays)
        points[i].set_data([positions[frame, i, 0]], [positions[frame, i, 1]])
        points[i].set_3d_properties([positions[frame, i, 2]])

        # Update planet trail (past positions)
        trails[i].set_data(positions[:frame, i, 0], positions[:frame, i, 1])
        trails[i].set_3d_properties(positions[:frame, i, 2])

    return points + trails

# Create the animation
ani = FuncAnimation(fig, update, frames=range(0, time_steps_count, 10), init_func=init, blit=True)

# Save the animation as a GIF using Pillow (instead of ImageMagick)
ani.save('planet_animation.gif', writer='pillow', fps=30)

plt.legend()
plt.show()